In [1]:
import os
import json
import requests
import time
from typing import Dict, List, Any, Tuple
from tqdm import tqdm
from datetime import datetime, timedelta
from collections import defaultdict

# LangChain imports
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Amadeus API client
from amadeus import Client, ResponseError

In [2]:
# Define top 20 cities and their codes
TOP_CITIES = {
    "NYC": "New York City",
    "LON": "London",
    "PAR": "Paris",
    "TYO": "Tokyo",
    "ROM": "Rome",
    "SIN": "Singapore",
    "HKG": "Hong Kong",
    "DXB": "Dubai",
    "SYD": "Sydney",
    "BCN": "Barcelona",
    "AMS": "Amsterdam",
    "IST": "Istanbul",
    "BER": "Berlin",
    "MAD": "Madrid",
    "BKK": "Bangkok",
    "VIE": "Vienna",
    "SEL": "Seoul",
    "LIS": "Lisbon",
    "CPH": "Copenhagen",
    "SFO": "San Francisco"
}

In [3]:
def initialize_amadeus():
    """Initialize Amadeus API client"""
    AMADEUS_CLIENT_ID = "1nsyVeNCT6PQtpqb1iG5MsIPWTkvN8bU"
    AMADEUS_CLIENT_SECRET = "GizAt9lxJi00tWG7"
    
    return Client(
        client_id=AMADEUS_CLIENT_ID,
        client_secret=AMADEUS_CLIENT_SECRET
    )

In [4]:
def fetch_amadeus_data(amadeus: Client, city_code: str) -> List[Document]:
    """Fetch all available data from Amadeus API for a city"""
    documents = []
    
    try:
        # Fetch flight offers
        next_month = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=city_code,
            destinationLocationCode='LON',  # Using London as example destination
            departureDate=next_month,
            adults=1,
            max=5
        )
        
        for flight in response.data:
            content = f"Flight from {city_code}:\n"
            for itinerary in flight.get('itineraries', []):
                for segment in itinerary.get('segments', []):
                    content += f"Carrier: {segment.get('carrierCode')}\n"
                    content += f"Departure: {segment.get('departure', {}).get('at')}\n"
                    content += f"Arrival: {segment.get('arrival', {}).get('at')}\n"
            
            documents.append(Document(
                page_content=content,
                metadata={"type": "flight", "city": city_code}
            ))
            
        # Fetch hotel information
        hotels = amadeus.reference_data.locations.hotels.by_city.get(
            cityCode=city_code
        )
        
        for hotel in hotels.data[:5]:  # Limit to 5 hotels per city
            content = f"Hotel in {city_code}:\n"
            content += f"Name: {hotel.get('name', 'N/A')}\n"
            content += f"Location: {hotel.get('address', {}).get('cityName', 'N/A')}\n"
            
            documents.append(Document(
                page_content=content,
                metadata={"type": "hotel", "city": city_code}
            ))
            
    except ResponseError as error:
        print(f"Error fetching Amadeus data for {city_code}: {error}")
    
    return documents

In [5]:
def fetch_wikivoyage_content(city: str) -> Dict[str, str]:
    """Fetch and parse travel information from Wikivoyage"""
    base_url = "https://en.wikivoyage.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "titles": city,
        "prop": "extracts",
        "explaintext": True,
    }
    
    try:
        response = requests.get(base_url, params=params)
        data = response.json()
        
        pages = data["query"]["pages"]
        page_id = list(pages.keys())[0]
        content = pages[page_id]["extract"]
        
        # Parse into sections
        sections = {
            "Understand": "",
            "Get in": "",
            "Get around": "",
            "See": "",
            "Do": "",
            "Buy": "",
            "Eat": "",
            "Sleep": "",
            "Stay safe": ""
        }
        
        current_section = "Understand"
        for line in content.split('\n'):
            if line.strip() in sections:
                current_section = line.strip()
            else:
                sections[current_section] += line + "\n"
        
        return sections
        
    except Exception as e:
        print(f"Error fetching Wikivoyage data for {city}: {str(e)}")
        return {}

In [6]:
def create_unified_document(city_code: str, city_name: str, wiki_data: Dict[str, str], 
                          amadeus_docs: List[Document]) -> Document:
    """Create a unified document combining both data sources"""
    
    # Extract Amadeus data
    flights = [doc.page_content for doc in amadeus_docs if doc.metadata["type"] == "flight"]
    hotels = [doc.page_content for doc in amadeus_docs if doc.metadata["type"] == "hotel"]
    
    # Create unified content
    content = f"""{city_name} ({city_code}) - Complete Travel Guide

    DESTINATION OVERVIEW
    {wiki_data.get('Understand', '')}

    GETTING THERE
    {wiki_data.get('Get in', '')}
    
    Current Flight Options:
    {chr(10).join(flights[:3]) if flights else 'No current flight data available'}

    LOCAL TRANSPORTATION
    {wiki_data.get('Get around', '')}

    ACCOMMODATIONS
    {wiki_data.get('Sleep', '')}
    
    Current Hotel Options:
    {chr(10).join(hotels[:3]) if hotels else 'No current hotel data available'}

    ATTRACTIONS
    {wiki_data.get('See', '')}

    ACTIVITIES
    {wiki_data.get('Do', '')}

    DINING
    {wiki_data.get('Eat', '')}

    SHOPPING
    {wiki_data.get('Buy', '')}

    SAFETY INFORMATION
    {wiki_data.get('Stay safe', '')}
    """
    
    return Document(
        page_content=content,
        metadata={
            "type": "unified_guide",
            "city_code": city_code,
            "city_name": city_name,
            "sources": ["wikivoyage", "amadeus"]
        }
    )

In [7]:
def build_unified_knowledge_base() -> FAISS:
    """Build unified knowledge base from both data sources"""
    unified_docs = []
    
    # Initialize Amadeus client
    amadeus = initialize_amadeus()
    
    print("Fetching and combining data for each city...")
    for city_code, city_name in tqdm(TOP_CITIES.items()):
        # Fetch Wikivoyage data
        wiki_data = fetch_wikivoyage_content(city_name)
        time.sleep(1)  # Be nice to the Wikivoyage API
        
        # Fetch Amadeus data
        amadeus_docs = fetch_amadeus_data(amadeus, city_code)
        
        # Create unified document
        if wiki_data or amadeus_docs:
            unified_doc = create_unified_document(
                city_code=city_code,
                city_name=city_name,
                wiki_data=wiki_data,
                amadeus_docs=amadeus_docs
            )
            unified_docs.append(unified_doc)
    
    print(f"Created {len(unified_docs)} unified documents")
    
    # Create vector store
    return FAISS.from_documents(
        documents=unified_docs,
        embedding=OpenAIEmbeddings()
    )

In [9]:
%run "temp.ipynb"

In [10]:
def main():
    """Main function to build knowledge base and initialize system"""
    try:
        # Build the unified knowledge base
        travel_db = build_unified_knowledge_base()
        print(f"Created unified knowledge base with {len(travel_db.index_to_docstore_id)} documents")
        
        # Create travel assistant with unified knowledge base
        travel_assistant = create_travel_assistant_graph()
        
        # Test the system with a sample query
        test_query = "Tell me about attractions in Paris"
        result = travel_assistant.invoke({"query": test_query})
        print("\nTest Query Result:")
        print(result.get("agent_response", "No response generated"))
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()

Fetching and combining data for each city...


 10%|█         | 2/20 [00:10<01:26,  4.80s/it]

Error fetching Amadeus data for LON: [400]



100%|██████████| 20/20 [02:09<00:00,  6.47s/it]


Created 20 unified documents
Created unified knowledge base with 20 documents

Test Query Result:
It seems there was an error while processing your request about attractions in Paris. I apologize for the inconvenience. 

For a general guide, Paris is filled with iconic attractions. Among the must-visit sites are the Eiffel Tower, the Louvre Museum, which houses thousands of works of art including the Mona Lisa, and the Cathedral of Notre Dame. You might also enjoy strolling along the Champs-Élysées, visiting the beautiful gardens of Luxembourg, or exploring the historic district of Montmartre.

To help you further, you could rephrase your question to focus on a specific type of attraction or area in Paris. For example, "What are the top museums to visit in Paris?" or "What are family-friendly activities in Paris?" This might help in getting a more specific response.
